# JUICE RPWI HF SID5 (PSSR1 surv): L1a QL -- 2023/11/18

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

# os.environ["CDF_LIB"] = "C:\\Program Files\\CDF_Distribution\\cdf39_0-dist\\lib"
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid5_lib as juice_sid5

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
unit_mode = 0                           # 0: sum    1: /Hz
cal_mode = 2                            # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 1                              # 0: linear   1:log  in frequency
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 10.0                        # background: 7.5   CAL: 10
p_raw_min = 4.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(unit_mode, cal_mode, cal, p_raw_max, p_raw_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

# *** Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID05_20231024-0046.cdf'
data_name = 'SID05_20231117-1603.cdf'
data_name = 'SID05_20231117-1611.cdf'

# 202310 -- FS
#data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202310_FS/cdf/'        # CDF data folder
#data_name = 'SID05_Seq01.cdf'

# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid5.juice_getdata_hf_sid5(cdf, cf)

date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_freq = data.EE.shape[1]
n_time = data.EE.shape[0]
n_time0 = n_time-1
print("data size:", data.EE.shape, n_time, n_freq)

In [ ]:
# Mode 
print("[Data Mode]")
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
N_component = 1
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
N_frequency = data.N_freq[0]   # U_selected[0]np.int16(data.B0_step[0] * data.B0_subdiv[0] / data.subdiv_reduction[0])
print("==> Component", N_component, "   Num-frequency", N_frequency, "  Num-sweep:", n_time)

# Data for checks

In [ ]:
Epoch_1d      = data.epoch.tolist()
freq_1d       = data.frequency[0].tolist()
freq1         = freq_1d[0:N_frequency]
freq_width_1d = data.freq_width[0].tolist()
freq_width1   = freq_width_1d[0:N_frequency]

EE_2d = data.EE.transpose()
# EE_power1 = data.EE[0][0:N_frequency]
# EE_power2 = data.EE[n_time0][0:N_frequency]
#
freq_width_2d = data.freq_width.transpose()
if unit_mode == 1:
    EE_2d = EE_2d / freq_width_2d / 1000 * data.subdiv_reduction[0]
    # EE_power1 = EE_power1 / freq_width1 / 1000 * data.subdiv_reduction[0]
    # EE_power2 = EE_power2 / freq_width1 / 1000 * data.subdiv_reduction[0]
    """
    print("frequency width:", freq_width1[0], data.subdiv_reduction[0])
    """

f_min = freq_1d[0]
f_max = freq_1d[N_frequency-1]
power_str = juice_cdf.power_label(cal_mode, unit_mode)

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(np.ravel(data.EE), '-r', linewidth=.5, label='EE')
ax1.set_yscale('log')
#
ax2.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
ax2.plot(np.ravel(data.freq_step*100),  '-g', linewidth=0.8, label='step*100')
ax2.plot(np.ravel(data.freq_width*100), ':b', linewidth=1.0, label='width*100')
ax3.plot(np.ravel(data.epoch), '.')

ax1.set_ylabel(power_str)
ax2.set_ylabel('frequency [kHz]')
ax3.set_ylabel('Date-Time')
ax3.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR1-Surv (SID-5)]  ' + data_name
ax1.set_title(title_label);  
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8);

"""
# range: X-axis
xlim=[0, N_frequency]
ax1.set_xlim(xlim)
ax2.set_xlim(xlim)
# range: Y-axis
ylim=[10**p_raw_min, 10**p_raw_max]
ax1.set_ylim(ylim)
"""
ylim=[f_min, f_max]
ax2.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## Spec

In [ ]:
# Sweep_num
n0 = 0
n1 = np.int16(n_time0/2)
n2 = n_time0
# X-range
f_min0 = f_min;  f_max0 = f_max
# Y-range
p_min0 = p_min;  p_max0 = p_max

fig, cal = plt.subplots(figsize=(12, 8))

cal.plot(freq1, data.EE[n0][0:N_frequency], '-r', linewidth=1.0, label='uu init')
cal.plot(freq1, data.EE[n1][0:N_frequency], '-g', linewidth=1.0, label='uu half')
cal.plot(freq1, data.EE[n2][0:N_frequency], '-b', linewidth=1.0, label='uu end')
cal.set_yscale('log')
if f_mode == 1:
    cal.set_xscale('log')

# Label
cal.set_ylabel(power_str)
cal.set_xlabel('Frequency [kHz]')
#
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3 
cal.set_title(title_date)
cal.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min0, f_max0];           cal.set_xlim(xlim)
# range: Y-axis
ylim=[10**p_min0, 10**p_max0];   cal.set_ylim(ylim)

plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

## FT

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
# Z-range
p_min0 = p_min;      p_max0 = p_max

fig2d = plt.figure(figsize=[12,8])
ax1 = fig2d.add_subplot(1, 1, 1)

# Y-axis
ax1.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EE')
ax1.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(power_str)

plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)
